In [1]:
import pandas as pd
from tqdm.auto import tqdm
import numpy as np

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', 300)

In [2]:
import json

with open("../data/fix_ipc_class_dict.json") as json_file:
    ipc_class_dict = json.load(json_file)

In [3]:
#df = pd.read_csv("../data/subest1_filtrato.csv")
df = pd.read_csv("../data/data_subset1.csv")

In [4]:
df.title = df.title.str.lower()
df.abstract = df.abstract.str.lower()
df.claims = df.claims.str.lower()
df.ipc_classes = df.ipc_classes.apply(lambda x: x.replace(",", ", ") if type(x)==str else "")

In [5]:
df.head()

,filename,filing_date,publication_date,priority_date,ipc_classes,assignee,inventors,docdb_family_id,title,abstract,claims
0,US2009216698_A1.txt,2008-02-22,2009-08-27,2008-02-22,"G06F15/18, G06N5/02","XEROX CORPORATIONANDREOLI, JEAN-MARCBOUCHARD, GUILLAUME","ANDREOLI, JEAN-MARCBOUCHARD, GUILLAUME",40999265,temporal events analysis employing tree induction,an events analysis method comprises: optimizing respective to a set of training data a set of branching transition likelihood parameters associating parent events of type k with child events of type k' in branching processes; inferring a most probable branching process for a set of input data co...,1. an events analyzer comprising: a probabilistic branching process learning engine configured to optimize respective to a set of training data a set of branching transition likelihood parameters associating parent events of type k with child events of type k′ in branching processes; and a proba...
1,US2020018760_A1.txt,2019-09-23,2020-01-16,2007-03-27,G01N33/574,IMMUNOVIA,"BORREBAECK, CARL, ARNE, KRISTERWINGREN, LARS, BERTIL, CHRISTER",39493874,protein signature/markers for the detection of adenocarcinoma,the present invention provides a method for determining the presence of pancreatic adenocarcinoma in an individual and/or for determining the survival time of an individual afflicted with pancreatic adenocarcinoma comprising the steps of: (a) providing a serum or plasma sample to be tested; and ...,1. a method for determining the presence of pancreatic adenocarcinoma in an individual comprising the steps of: a) providing a serum or plasma sample to be tested; b) determining a protein signature of the test sample by measuring the presence and/or amount in the test sample of one or more prot...
2,EP3340117_A1.txt,2017-11-15,2018-06-27,2016-12-20,"G06N3/04, G06N3/063",INTEL CORPORATION,"LIN, TSUNG-HAN",60327233,unsupervised learning using neuromorphic computing,"a spiking neural network (snn) is implemented on a neuromorphic computers and includes a plurality of neurons, a first set of the plurality of synapses defining feed-forward connections from a first subset of the neurons to a second subset of the neurons, a second subset of the plurality of syna...","1. a method comprising: defining a spiking neural network (snn) comprising a plurality of artificial neurons interconnected by a plurality of artificial synapses, wherein the snn is to comprise: a first subset of the plurality of synapses to define feedforward connections from a first subset of ..."
3,WO2019070570_A1.txt,2018-10-01,2019-04-11,2017-10-06,A61B5/103,TELLUS YOU CARE,"HSU KEVINCOKE, TANIA",65993331,non-contact activity sensing network for elderly care,"determining a physical state of a person includes detecting positions of different portions of the person, transforming detected positions of the person into a point cloud having a density that varies according to movement of each of the portions, correlating movement and position data from the ...","1. a method of determining a physical state of a person, comprising:detecting positions of different portions of the person;transforming detected positions of the person into a point cloud having a density that varies according to movement of each of the portions;correlating movement and positio..."
4,US2016259635_A1.txt,2015-03-04,2016-09-08,2015-03-04,"G06F15/18, G06F17/27, G06F8/65, G06F8/71, G06F9/44, G06F9/445, G06N99/00",IBM (INTERNATIONAL BUSINESS MACHINES CORPORATION),"EL MAGHRAOUI, KAOUTARJANN, JOEFONPATTNAIK, PRATAP C.PICKOVER CLIFFORD A.",56849635,software patch management incorporating sentiment analysis,"a method and system are provided. the method includes generating, by a machine-based sentiment prediction generator, respective machine-determined sentiment predictions for each of a plurality of software patches using sentiment analysis. the method further includes setting, by a sentiment-based...","1. 1-18. (canceled)19. a computer program product for softwar

In [6]:
len(df)

216986

<h3>Keyword della professoressa:</h3>

<img src="keyword prof.png" style="width:800px;"/>

In [40]:
keywords_other = []
"""["molecule",
                  "particle", 
                  "pharmacological",
                  "technolog",
                  "nano(-| )?tech(nolog)?",
                  "alter(ing)?",
                  "nano",
                  "nano(-| )?bot",
                  "nano(-| )?emulsion",
                  "nano(-| )?particle",
                  "nano(-| )?tube",
                  "nano(-| )?sphere",
                  "quantum",
                  "graphene",
                  "cloud",
                  "software",
                  "product"
                 ]"""

"""
Possiamo cercare i vincoli espressi dalla professoressa sia in "stile pandas" 
    (cercare documenti che contengono sia "body" che "care": df[(df.body >= 1) & (df.care >= 1)])
    
    che tramite l'uso delle espressioni regolari per implmentare l'operatore che l'Ing Ghironi 
    ha chiamto NEAR. Ad esempio per cercare "skin NEAR care" possiamo aggiungere alla lista sotto
    "lip(\w|\W){1,20}?stick". Agli estremi ci sono le 2 parole in cui siamo interessati, 
    "(\w|\W){1,20}?" serve invece a dire "permetti la presenza tra 
    0 e 20 caratteri o numeri tra le 2 parole". Se volete fare dei test: https://pythex.org/
"""

keywords_cosmetics = ["cosmetic",
                      "beauty", 
                      "body", 
                      "skin", 
                      "nail", 
                      "fingernail", 
                      "cleans(e|ing)", # 
                      "manicur(e|ing)", #manicure and manicuring
                      "pedicur(e|ing)"
                      "varnish", 
                      "polish",
                      "nail(s| |-)*polish",
                      "lip(s)?",
                      "lip( |-)?stic( |k)+",
                      "blush",
                      "mascara",
                      "make(s| |-)*up",#makes up, make-up
                      "cream",
                      "soap",
                      "toothpaste",
                      "sunscreen",
                      "lotion",
                      "deodorant",
                      "care",
                      "lip(s| |-)*(care|polish)",
                      "skin(s| |-)*(care|polish)",
                      "skin(s| |-)*(care|polish)",
                      "nail(s| |-)*(care|polish)",
                      "baby(s| |-)*(care|polish)",
                      "anti(-)?ag(e|ing)",
                      "colour|color",
                      "finger"
                     ]

## Analisi Per keyword (solo titolo)
#### Vedi slides Ing. Ghironi

In [41]:
for key in tqdm(keywords_cosmetics):
    df[key] = df.title.str.count(key).fillna(0) + df.abstract.str.count(key).fillna(0) + df.claims.str.count(key).fillna(0)
    
for key in tqdm(keywords_other):
    df[key] = df.title.str.count(key).fillna(0) #+ df.abstract.str.count(key).fillna(0) + df.claims.str.count(key).fillna(0)

  0%|          | 0/31 [00:00<?, ?it/s]

0it [00:00, ?it/s]

In [42]:
df["total_cosmetics"] = 0
for key in keywords_cosmetics:
    if key == "body" or key == "lip(s)?": continue
    df["total_cosmetics"] += df[key]
    
df["total_other"] = 0
for key in keywords_other:
    df["total_other"] += df[key]

In [11]:
df_selected = pd.DataFrame()

In [12]:
keywords_cosmetics

['cosmetic',
 'beauty',
 'body',
 'skin',
 'nail',
 'fingernail',
 'cleans(e|ing)',
 'manicur(e|ing)',
 'pedicur(e|ing)varnish',
 'polish',
 'nail(s| |-)*polish',
 'lip(s)?',
 'lip( |-)?stic( |k)+',
 'blush',
 'mascara',
 'make(s| |-)*up',
 'cream',
 'soap',
 'toothpaste',
 'sunscreen',
 'lotion',
 'deodorant',
 'care',
 'lip(s| |-)*(care|polish)',
 'skin(s| |-)*(care|polish)',
 'skin(s| |-)*(care|polish)',
 'nail(s| |-)*(care|polish)',
 'baby(s| |-)*(care|polish)',
 'anti(-)?ag(e|ing)',
 'colour|color']

##### cosmetic

In [13]:
keyword = keywords_cosmetics[0]
keyword

'cosmetic'

In [14]:
df[df[keyword] > 0][["title", "ipc_classes","abstract", "claims"]]

,title,ipc_classes,abstract,claims
8593,"system for beauty, cosmetic, and fashion analysis","G06K9/00, G06K9/66, G06T11/00, G06T11/60, G06T3/00","a system and method are provided to detect, analyze and digitally remove makeup from an image of a face. an autoencoder-based framework is provided to extract attractiveness-aware features to perform an assessment of facial beauty.","1. a system for providing an assessment of facial attractiveness, comprising: one or more processors and memory, including a first auto-encoder trained with one or more visual descriptors of a first set of faces and a second auto-encoder trained with one or more visual descriptors of a second se..."
17489,"system for beauty, cosmetic, and fashion analysis","G06K9/00, G06K9/66, G06T11/00, G06T11/60, G06T3/00","a system and method are provided to detect, analyze and digitally remove makeup from an image of a face. an autoencoder-based framework is provided to extract attractiveness-aware features to perform an assessment of facial beauty.","1. a system for analyzing an image of a human face for the presence of makeup, comprising: one or more processors and memory, including a dataset comprising images of human faces, the images comprising facial images of multiple human subjects, and including multiple images associated with a sing..."
24039,apparatus and method for determining cosmetic skin attributes,"A45D44/00, A61B5/00, A61B5/103, G06K9/00, G06K9/20, G06K9/46, G16H50/20, G16H50/30, G16H50/70","a method of determining a cosmetic skin attribute of a person is provided. the method includes obtaining a color channel image of a person's skin, analyzing the color channel image with a computer using entropy statistics to obtain an entropy value, and then determining a cosmetic skin attribute...","1. a method of determining a cosmetic skin attribute of a person, the method comprising: a) obtaining at least one color channel image comprising at least one portion of skin of the person; b) analyzing the at least one color channel image using entropy statistics to obtain an entropy value; and..."
35892,inspection and cosmetic grading system and method,"A24C5/34, B61L23/04, G01B11/02, G01N21/88, G06T7/00, G06T7/521, G06T7/62, H04N7/18",a system and method for inspection and cosmetic grading of objects is provided. camera and lighting assemblies capture images of an object and create a 2d composite image which is processed by an image processing module with a deep learning machine algorithm to detect surface defects in the obje...,"what is claimed is:1. a device for inspecting and evaluating object surface defects,comprising: a computing device with one or more processors and memory; an automated control system controlled by a processor, operativelyconfigured to receive instructions from a control module comprising compute..."
38540,cosmetic cleaning product,"A45D34/04, A61H15/02, A61H7/00, A61K8/00, A61K8/02, A61K8/31, A61K8/37, A61K8/42, A61K8/44, A61Q19/10, A61Q5/02","the invention is a cosmetic cleaning product, comprising a massage applicator comprising a container body and a massage device having at least one massage pin device, the pin device having at least one first surface which is configured to be in contact with the surface to be massaged, wherein a ...","1. a cosmetic cleaning product, comprising: a massage applicator comprising a container body and a massage device having (a) at least one massage pin device, said pin device having at least one first surface which is configured to be in contact with the surface to be massaged, wherein a force ca..."
39592,cosmetic affinity indexing,"G06Q30/02, G06Q30/06",methods and systems are disclosed for helping a consumer make a cosmetic product purchasing decision. a cosmetic service may maintain information reflective of compatibility between cosmetic products and personal characteristics of a plurality of subjects. the cosmetic service may also collect p...,"1. a method of helping a consumer make a cosmetic prod

In [18]:
#escludiamo tutti i sistemi per "raccomandare un prodotto"

df_candidate = df[(df[keyword] > 0) 
                  & (df.ipc_classes.str.count("(^| )A")>1)][["title", "ipc_classes","claims"]] 
#Questo seleziona anche i patent relativi all'AR, che poi vanno riesaminati a mano

df_candidate

,title,ipc_classes,claims
24039,apparatus and method for determining cosmetic skin attributes,"A45D44/00, A61B5/00, A61B5/103, G06K9/00, G06K9/20, G06K9/46, G16H50/20, G16H50/30, G16H50/70","1. a method of determining a cosmetic skin attribute of a person, the method comprising: a) obtaining at least one color channel image comprising at least one portion of skin of the person; b) analyzing the at least one color channel image using entropy statistics to obtain an entropy value; and..."
38540,cosmetic cleaning product,"A45D34/04, A61H15/02, A61H7/00, A61K8/00, A61K8/02, A61K8/31, A61K8/37, A61K8/42, A61K8/44, A61Q19/10, A61Q5/02","1. a cosmetic cleaning product, comprising: a massage applicator comprising a container body and a massage device having (a) at least one massage pin device, said pin device having at least one first surface which is configured to be in contact with the surface to be massaged, wherein a force ca..."
54029,real time monitoring of cosmetic laser aesthetic skin treatment procedures,"A61B18/20, A61B34/32, A61N5/06",what we claim is:1. a cosmetic method of treating skin tissue with a source of treatment light comprising:providing a source of treatment light along an optical axis;providing one or more sources of illumination light along the optical axis;providing one or more sensors along the optical axis;pr...
65842,simulation of effects of cosmetic products using a three-dimensional facial image,"A45D44/00, A61B5/103, A61K8/00, A61Q90/00, G01J3/52, G06F19/00, G06Q10/00, G06T1/00, G06T11/00, G06T17/40","1. method of enabling an analysis using a three-dimensional facial image, the method comprising: constructing the three-dimensional facial image using at least one captured image of a subject's face; and simulating use of an aesthetic feature on the three-dimensional facial image and/or processi..."
70595,method of evaluating cosmetic products on a consumer with future predictive transformation,"A45D44/00, A61B5/00, A61B5/103, A61B5/107, G06T1/00, G06T11/00, G06T11/60","1. a method for demonstrating effectiveness of a cosmetic product on a consumer's body parts when applied over a period of time, the method comprising: (i) selecting a cosmetic product for trial on the consumer's body and identifying a body feature being observed over the time; (ii) capturing an..."
79795,simulation of effects of cosmetic products using a three-dimensional facial image,"A45D44/00, A61B5/103, A61K8/00, A61Q90/00, G01J3/52, G06F19/00, G06Q10/00, G06T1/00, G06T11/00, G06T17/40","method of enabling an analysis using a three-dimensional facial image, the method comprising: constructing the three-dimensional facial image using at least one captured image of a subject's face; and simulating use of an aesthetic feature on the three-dimensional facial image and/or processing ..."
134898,system for dispensing a cosmetic product,"A45D34/04, A45D40/24, A45D44/00, A61K8/19, A61K8/73, A61K8/81, A61Q1/02, A61Q19/00","1. a system for dispensing a product, comprising a dispenser that receives at least two cartridges that each have a reservoir comprising a base product, a first base product comprising at least 0.1% by mass of particles exhibiting a difference in density with a medium which contains them, the pa..."
169482,cosmetic cleaning product,"A45D34/04, A61H15/02, A61H7/00, A61K8/00, A61K8/02, A61K8/31, A61K8/37, A61K8/42, A61K8/44, A61Q19/10, A61Q5/02","that which is claimed:1. a cosmetic cleaning product, comprising: a massage applicator comprising a container body and a massage device having (a) at least one massage pin device, said pin device having at least one first surface which is configured to be in contact with the surface to be massag..."


In [21]:
df_selected = pd.concat([df_selected, df_candidate])

##### beauty (da provare combinazioni)

##### body (da provare combinazioni

In [33]:
keyword = keywords_cosmetics[2]
keyword

'body'

##### skin (da provare combinazioni)

In [34]:
keyword = keywords_cosmetics[3]
keyword

'skin'

##### skin (da provare combinazioni)

In [37]:
keyword = keywords_cosmetics[4]
keyword

'nail'

In [38]:
keyword = keywords_cosmetics[5]
keyword

'fingernail'

### Controllo 
##### IPC A45D+

HAIRDRESSING OR SHAVING EQUIPMENT ; EQUIPMENT FOR COSMETICS OR COSMETIC TREATMENTS, e.g. FOR MANICURING OR PEDICURING

In [44]:
df_candidate = df[df.ipc_classes.str.count("(^| )A45D")>0][["title", "ipc_classes","claims"]] 
df_candidate

,title,ipc_classes,claims
3240,system for dispensing at least one makeup product and method for dispensing and evaluating makeup,"A45D40/00, A45D40/26, A45D44/00, B01F13/10, B01F15/02, B01F5/06, G01J3/46","1. a system for dispensing at least one makeup product and that can be manipulated in one hand, the system comprising: a cup; and a dispenser for filling the cup from a bottom of the cup with at least two base products, wherein: the cup is secured to the dispenser at least while the cup is being..."
11902,image processing apparatus and method for processing image,"A45D33/38, A45D34/04, A45D44/00, B41M5/00, G06K9/00, G06T7/73, H04N1/00","1. an image processing apparatus that generates image data used to print images on a sheet appliable to skin, the image processing apparatus comprising: processing circuitry configured to: determine a makeup article image whose target position on a face is determined in accordance with facial fe..."
20694,image processing apparatus and method for processing image,"A45D33/38, A45D34/04, A45D44/00, B41M5/00, G06K9/00, G06T7/73, H04N1/00","1. an image processing apparatus that generates image data used to print images on a sheet appliable to skin, the image processing apparatus comprising: an image determiner that determines a makeup article image whose target position on a face is determined in accordance with facial feature poin..."
20938,automatic nail polish application system and method,"A45D29/00, A45D29/11, A45D29/12, A45D29/14, A45D34/04","1. a system for automatically polishing a nail of a user including:an end-effector havinga cartridge receiving unit,a cartridge including nail polish, anda nozzle;a sensor, to generate target signals from a target location;a nail determination unit, receiving said target signals, to automaticall..."
22088,grooming instrument configured to monitor hair loss,"A45D24/00, A45D44/00, A61B5/00, A61B5/11","1. an instrumented comb having a handle, the instrumented comb comprising:a plurality of bristles protruding from the handle;a power source; anda controller enclosed in the handle receiving signals from:a contact sensor to detect contact with a scalp, anda magnetic compass to determine location ..."
29214,makeup evaluation system and operation method thereof,"A45D44/00, A45D44/02, G06K9/00, G06K9/22, G06K9/46, G06K9/62, G06Q30/02, G06Q30/06, G06Q50/10, G06T7/00, G09B19/00, H04N5/225","1. a makeup evaluation system comprising: a mobile terminal for photographing a facial image and transmitting the photographed facial image to a makeup server; and the makeup server for storing makeup score data, when receiving the facial image from the mobile terminal, detecting at least one fa..."
53114,portable apparatus for supporting a hand held hair dryer,"A45D20/12, A45D20/14, F16M11/00, F16M13/02","1. a portable hair dryer support apparatus, comprising: a hair dryer holding structure for holding a hair dryer; at least one vertical support element pivotally and directly connected at a bottom end to the hair dryer holding structure via a hinge having an axis of rotation perpendicular to the ..."
53136,improvements in and relating to hair irons,"A45D1/06, A45D1/28",1. an electric hair iron comprising at least one heating element and control means comprising a temperature sensor and a control circuit and wherein the control means is arranged such that power is only supplied to the heating element or elements when the temperature sensed by the sensor is at o...
61966,complete compact,"A45D33/00, A45D33/26, A45D44/00, G01N1/22, G01N1/24","i claim:1. a method for determining aroma comprising: (a) providing at least one means of aroma sensing capable of approximately discerning the presence or absence of at least one characteristic of the aroma, (b) sampling aroma with said aroma sensing means, (c) discerning said aroma characteris..."
63517,disposable single use applicator assembly with a chemical composition,"A45D37/00, A45D40/00, A45D40/26, A61K8/04, A61Q19/00","i claim:1. 

##### IPC C11B9/00 (riportato nelle slide come C11B/009)

In [57]:
df_candidate = df[df.ipc_classes.str.count("(^| )C11B9/00")>0][["title", "ipc_classes","claims"]] 
df_candidate

,title,ipc_classes,claims
5337,method of making perfumed goods,"C11B9/00, C11D3/50","1. a method comprising: a.) using an odt value determined by mathematical modelling to select one or more perfume raw materials; b.) combining and/or processing said one or more perfume raw materials with one or more additional materials to form a product.2. the method of claim 1, wherein said p..."
8543,olfactory signature and odorant mixture having the same,"C11B9/00, C11B9/02, G01N33/00, G06F17/30, G06F19/00","1. an odorant mixture, comprising n odorant components wherein n equals at least 20, each odorant component being characterized by a multidimensional vector of attributes, such that the mixture is characterized by n respective multidimensional vectors, wherein a z score of an average of characte..."
42547,olfactory signature and odorant mixture having the same,"C11B9/00, C11B9/02, G01N33/00, G06F17/30, G06F19/00","1. an odorant mixture, comprising n odorant components wherein n equals at least 20, each odorant component being characterized by a multidimensional vector of attributes, such that the mixture is characterized by n respective multidimensional vectors, wherein a z score of an average of characte..."
48466,olfactory signature and odorant mixture having the same,"C11B9/00, C11B9/02, G01N33/00, G06F17/30, G06F19/00","1. a method of determining an ability of an odorant mixture having n odorant components to mask other odors, the method being executable by a data processor and comprising: accessing a database having a group of m odorant components; for each odorant component, obtaining a multidimensional vecto..."
106943,method of making perfumed goods,"A61K8/33, A61K8/34, A61K8/35, A61K8/36, A61K8/368, A61Q15/00, A61Q19/10, C11B9/00, G06F17/10, G06F19/00, G06Q99/00","1. a method comprising: a.) using a malodour reduction value determined by mathematical modelling to select one or more perfume raw materials; b.) combining and/or processing said one or more perfume raw materials with one or more additional materials to form a product.2. the method of claim 1, ..."
135822,method of resolving allergens in perfume ingredients,"B01J20/26, B01J20/281, C11B9/00, G01N30/46, G01N30/86, G16C20/20","1. a method of resolving defined known or suspected allergens in a complex mixture of perfume ingredients using two-dimensional gas chromatography (2d-gc), wherein the combination of first and second stationary phases is selected on the basis of a clustering analysis of defined known or suspecte..."
174990,method of making perfumed goods,"A61L9/01, A61Q19/00, C11B9/00, C11D3/22, C11D3/50",what is claimed is:1. a method comprising:a.) using a malodour reduction value determined by mathematical modelling toselect one or more perfume raw materials;b.) combining and/or processing said one or more perfume raw materials with one or more additional materials to form a product. 2. the me...
175028,method of making perfumed goods,"A61K8/33, A61K8/34, A61K8/35, A61K8/37, A61Q13/00, A61Q15/00, A61Q19/10, C11B9/00, C11D11/00, C11D3/50, G06F17/50","1. a method comprising: a.) using a malodour reduction value determined by mathematical modelling to select one or more perfume raw materials; b.) combining and/or processing said one or more perfume raw materials with one or more additional materials to form a product.2. the method of claim 1, ..."
210149,"methods for identifying, compounds identified and compositions thereof","A23L27/00, C11B9/00","1. a flavor/fragrance composition, comprising an effective amount of one or more compounds selected from tables 1-42.2. the flavor/fragrance composition of claim 1, wherein the composition is formulated as a spray, lotion, foam, gel, suspension, or emulsion.3. the flavor/fragrance composition of..."


In [ ]:
##### VECCHIO CODICE:

In [49]:
df_selected[["filename"]].to_excel("selected_patents_filename_BIG.xlsx", index=False, header=False)

In [50]:
df_selected.to_excel("selected_patents_BIG.xlsx", index=False)

In [417]:
for ipc in very_interesting_ipc_classes:
    try:
        print(f"{ipc}: {ipc_class_dict[ipc].split(' - ')[-1]}")
    except:
        print(f"{ipc}: none")

A45D44/00: A45D44/00: Other cosmetic or personal care articles, e.g. for hairdressers' rooms + 
A45D29/14: A45D29/14: Manicuring or pedicuring implements + motor-driven
A45D29/00: A45D29/00: Manicuring or pedicuring + 
A61K8/00: A61K8/00: Cosmetics or similar toilet preparations + 
A61Q1/00: A61Q1/00: Make-up preparations; Body powders; Preparations for removing make-up + 
A61Q15/00A61Q19/10: none
C11B9/00: C11B9/00: Essential oils; Perfumes + 
A23L27/10: A23L27/10: Spices; Flavouring agents or condiments; Artificial sweetening agents; Table salts; Dietetic salt substitutes; Preparation or treatment thereof + Natural spices, flavouring agents or condiments; Extracts thereof
A45D34/04: A45D34/04: Containers or accessories specially adapted for handling liquid toilet or cosmetic substances, e.g. perfumes + Appliances specially adapted for applying liquid, e.g. using roller or ball
A45D31/00: A45D31/00: Artificial nails + 
A46B9/04: A46B9/04: Arrangements of the bristles in the brush body